In [1]:
!gpustat -cu

n100.mcl.weizmann.ac.il  Fri Jan 28 11:29:32 2022  455.32.00
[0] Tesla V100-PCIE-16GB | 25'C,   0 % |     0 / 16160 MB |
[1] Tesla V100-PCIE-16GB | 25'C,   0 % |     0 / 16160 MB |
[2] Tesla V100-PCIE-16GB | 61'C,  99 % | 16127 / 16160 MB | odeliam:python(16123M)
[3] Tesla V100-PCIE-16GB | 60'C,  97 % | 16145 / 16160 MB | odeliam:python(16141M)


In [2]:
%env CUDA_VISIBLE_DEVICES 1

env: CUDA_VISIBLE_DEVICES=1


In [3]:
import itertools
import math
import os
import pickle
import random
import time
from pprint import pprint
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from bm3d import bm3d
from image import imread, imwrite, imshow, to_numpy


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', DEVICE)

device: cuda


In [4]:
def np2pt(arr):
    return torch.tensor(arr).permute(2, 0, 1).unsqueeze(0).contiguous()

def pt2np(tensor):
    return to_numpy(tensor.squeeze(0).permute(1, 2, 0))

def bm3d_denoise(input, sigma):
    return np2pt(bm3d(pt2np(input), sigma)).to(device=input.device, dtype=input.dtype)

def imread2pt(path, d=None, **kwargs):
    image = imread(path, **kwargs)
    image = np2pt(image)
    # image = torch.tensor(imread(path, **kwargs)).permute(2, 0, 1).unsqueeze(0)
    if d is not None:
        image = shave_image(image, d)
    return image.contiguous()

def get_psnr(a, b, bounds=(0., 1.)):
    mse = F.mse_loss(a, b)
    return 20 * math.log10(bounds[1] - bounds[0]) - 10 * mse.log10()


In [5]:
import Resizer
from PIL import Image
def get_pyramids_aux(img_torch, num_layers,ratio, print_):
    max_layer = num_layers
    pyr = []
    ratio = ratio
    if isinstance(ratio, float) or isinstance(ratio, int):
        ratio = (ratio, ratio)
    curr_img = img_torch
    pyr.append(curr_img)
    for j in range(max_layer):
        if print_:
            print(curr_img.shape)
            imshow(curr_img)
        rsizer = Resizer.Resizer(curr_img.shape, ratio, [1,img_torch.shape[1], np.ceil(img_torch.shape[2]*(ratio[0])**(j+1)), np.ceil(img_torch.shape[3]*(ratio[1])**(j+1))]).cuda()
        curr_img = rsizer(curr_img)
        pyr.append(curr_img)
        
    return pyr

def get_pyramid(file, degradation, sigma, num_layers, ratio, print_=True):

    gt = np.expand_dims(imread(file),0)
    gt = torch.tensor(gt).contiguous().permute(0,3,1,2).detach().cuda()
            
    return get_pyramids_aux(gt, num_layers,ratio, print_)

In [1]:
import torch

In [ ]:
k = torch.arange(30).reshape(2, 5, 3)
m = torch.ones((2, 5, 3), dtype=torch.bool)
m[0, 0, 1] = False
m[0, 0, 1] = False

In [11]:
DIV = 100

from typing import Optional, Sequence

import fold
import new_utils as utils

from fold import unfold2d, fold2d, view_as_column, view_as_image

def _unfold2d_to_column(image, *args, **kwargs):
    unfolded = fold.unfold2d(image, *args, **kwargs)
    return fold.view_as_column(fold.unfold2d(image))[0]

def pnn(queries: torch.Tensor, keys: torch.Tensor, values: torch.Tensor, 
        masks: Optional[torch.Tensor] = None, patch_size: int = 7, alpha: float = 10.0, 
        reduce='weighted_mean'):
    query_patches = fold.unfold2d(queries)
    query_patches_column, query_patches_size = fold.view_as_column(query_patches)        
    key_patches_column, _ = fold.view_as_column(fold.unfold2d(keys))
    value_patches_column, _= fold.view_as_column(fold.unfold2d(values))
    if masks is not None:
        mask_patches_column, _ = fold.view_as_column(fold.unfold2d(masks))
        unmasked_patches = mask_patches_column.sum(dim=2) > mask_patches_column.shape[2] - 0.5
        key_patches_column = key_patches_column[:, :, unmasked_patches]
        value_patches_column = value_patches_column[:, :, unmasked_patches]
    dist = utils.compute_dist_matrix(query_patches_column, key_patches_column)  # MSE
    dist /= dist.min(dim=0, keepdim=True)[0] + alpha
    indices = dist.argmin(1)
    out_patches_column = F.embedding(indices, value_patches_column)  # .unsqueeze(0).transpose(1,2)
    out_patches = view_as_image(out_patches_column, query_patches_size)
    return fold2d(out_patches, reduce=reduce)

# def build_image_bidirectional_weighted(input_img, index_imgs, ref_imgs, patch_size=7, eps=1):
#     unfold = torch.nn.Unfold(kernel_size=(patch_size,patch_size))
#     in_patches = unfold2d(input_img, use_padding=False, use_view=True)
#     in_patches, patches_shape = view_as_column(in_patches)
#     for i, (ind_img, ref_img) in enumerate(zip(index_imgs, ref_imgs)):
#         if i == 0:
#             index_patches = unfold2d(ind_img, use_padding=False)
#             ref_patches = unfold2d(ref_img, use_padding=False)
#         else:
#             index_patches = torch.cat([index_patches, unfold2d(ind_img, use_padding=False)],dim=-1)
#             ref_patches = torch.cat([ref_patches, unfold2d(ref_img, use_padding=False)], dim=-1)
            
#     if in_patches.shape[-1]*ref_patches.shape[-1] > 14000**2:
#         for j in range(DIV):
#             start_patch = j*(math.ceil(in_patches.shape[-1]/DIV))
#             end_patch = min((j+1)*(math.ceil(in_patches.shape[-1]/DIV)), in_patches.shape[-1])
#             dist_mat = _calc_dist_l2(in_patches[:,:,start_patch:end_patch].permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
#             dist_mat /= patch_size*patch_size*3
#             if start_patch < end_patch:
#                 dist_mat = dist_mat/(dist_mat.min(0, keepdim=True)[0] + eps)
#             if j == 0:
#                 ind = dist_mat.argmin(1)
#             elif start_patch < end_patch:
#                 ind = torch.cat([ind, dist_mat.argmin(1)])

#     else:
#         dist_mat = _calc_dist_l2(in_patches.permute(0,2,1).squeeze(0), index_patches.squeeze(0).permute(1,0))
#         dist_mat /= patch_size*patch_size*3
#         dist_mat = dist_mat/(dist_mat.min(0, keepdim=True)[0] + eps)
#         ind = dist_mat.argmin(1)
#     out_patches = F.embedding(ind, ref_patches.squeeze(0).permute(1,0)).unsqueeze(0).transpose(1,2)
#     out_patches = view_as_image(out_patches, patches_shape)
#     return fold2d(out_patches, reduce='weighted_mean', use_padding=False)



In [7]:
# clean functions
import resize_right

def gpnn(pyr: list, query: torch.Tensor, noise_std: float = 0.75, alpha: float = 10.0,
        downscale_ratio: float = 0.75, patch_size: int = 7, reduce: str = 'weighted_mean'):
  start = time.time()
  query += torch.randn_like(query)*noise_std
  for l in range(len(pyr),-1,-1): 
    num_iters = 1 if l == top_level else 10
    for k in range(num_iters):
        if l == top_level:
            new_im = pnn(query, key=[pyr[l]] ,value=[pyr[l]], patch_size=patch_size, reduce=reduce)
        else:
            blurred = resize_right.resize(pyr[l+1], 1/downscale_ratio, pyr[l].shape)
            # rsizer = Resizer.Resizer(pyr[l+1].shape, 1/downscale_ratio, pyr[l].shape).to(device=device)
            query = pnn(query, key=[blurred] ,value=[pyr[l]], patch_size=patch_size, reduce=reduce)
    if l > 0:
        query = resize_right.resize(pyr[l], 1/downscale_ratio, pyr[l-1].shape)
        # rsizer = Resizer.Resizer(pyr[l].shape, 1/downscale_ratio, pyr[l-1].shape).to(device=device)
  print('Total time: %.3f[s]' % (time.time() - start,))
  return query

def image_generation(input_path: str, noise_std: float = 0.75, alpha: float = 10.0,
                    patch_size: int = 7, reduce: str = 'weighted_mean', 
                    downscale_ratio: float = 0.75, num_levels: int = 9):
    pyr = make_pyramid(imread(input_path), num_levels, downscale_ratio)
    return gpnn(pyr, query=pyr[-1], noise_std=noise_std, alpha=alpha, 
        downscale_ratio=downscale_ratio, patch_size=patch_size, reduce=reduce)

def image_editing(input_path: str, input_path_edited: str, noise_std: float = 0.75, 
                    alpha: float = 10.0, device: str = 'cuda', 
                    patch_size: int = 7, reduce: str = 'weighted_mean', 
                    downscale_ratio: float = 0.75, num_levels: int = 9):
    pyr = make_pyramid(imread(input_path), num_levels, downscale_ratio)
    pyr_edited = make_pyramid(imread(input_path_edited), num_levels, downscale_ratio)
    return gpnn(pyr, query=pyr_edited[-1], noise_std=noise_std, alpha=alpha, 
        downscale_ratio=downscale_ratio, device=device, patch_size=patch_size,
        reduce=reduce)

def retargeting(input_path: str, noise_std: float = 0.0, 
                alpha: float = 1e-3, device: str = 'cuda', 
                patch_size: int = 7, reduce: str = 'weighted_mean', 
                downscale_ratio: float = 0.8, num_levels: int = 9,
                retargeting_ratio: tuple = (0.75, 0.75), gradual: bool = False):
    pyr = make_pyramid(imread(input_path), num_levels, downscale_ratio)
    if gradual:
        step_size = [0.9, 0.9]
        if retargeting_ratio[0] >= 1:
            step_size[0] = 1.1
        if retargeting_ratio[1] >= 1:
            step_size[0] = 1.1
        num_steps = math.floor(max(math.log(ratio[0])/math.log(step_size[0]), 
                                math.log(ratio[1])/math.log(step_size[1])))
        step_size[0] = 10**(math.log10(ratio[0])/num_steps)
        step_size[1] = 10**(math.log10(ratio[1])/num_steps)



    resized = resize_right.resize(pyr[0], retargeting_ratio)
    top_level = math.floor(min(math.log(min_size/new_im.shape[-2])/math.log(downscale_ratio), 
                math.log(min_size/new_im.shape[-1])/math.log(downscale_ratio)))
    retargeted_pyr = make_pyramid(resized, top_level, downscale_ratio)        
    query = new_pyr[top_level].clone()
    return gpnn(pyr, query=query, noise_std=noise_std, alpha=alpha, 
        downscale_ratio=downscale_ratio, device=device, patch_size=patch_size,
        reduce=reduce)



In [14]:
# Image Generation


# def new_image_generation(src_pyr, dst_pyr, patch_size=7, top_level=9, noise_std=0.75):
#     start = time.time()
#     new_im = dst_pyr[top_level] + torch.randn_like(dst_pyr[top_level])*noise_std
#     print(new_im.shape)
#     for l in range(top_level,-1,-1): 
#         num_iters = 1 if l == top_level else 10
#         for k in range(num_iters):
#             rsizer = Resizer.Resizer(src_pyr[l+1].shape, 4/3, src_pyr[l].shape).cuda()
#             if l == top_level:
#                 new_im = build_image_weighted(new_im, [src_pyr[l]] ,[src_pyr[l]], patch_size=7)
#             else:
#                 new_im = build_image_weighted(new_im, [rsizer(src_pyr[l+1])] ,[src_pyr[l]], patch_size=7)
# #         imshow(torch.cat([new_im,pyr[l]], dim=-1))
#         if l > 0:
#             rsizer = Resizer.Resizer(dst_pyr[l].shape, 4/3, dst_pyr[l-1].shape).cuda()
#             new_im = rsizer(new_im)
# #     imshow(new_im)
#     print('Total time: %.3f[s]' % (time.time() - start,))
#     return new_im


# def edit(src_pyr, dst_pyr, mask_size, patch_size=7, top_level_max=9):
#     start = time.time()
#     top_level = int(np.floor(np.log(patch_size/mask_size)/np.log(3/4)))
#     top_level = min(top_level_max, top_level)
#     new_im = dst_pyr[top_level]
#     for l in range(top_level,-1,-1): 
#         num_iters = 1 if l == top_level else 10
#         new_im = new_im + torch.randn_like(new_im)*(0.4**(10 -l))
#         for k in range(num_iters):
#             rsizer = Resizer.Resizer(src_pyr[l+1].shape, 4/3, src_pyr[l].shape).cuda()
#             if l == top_level:
#                 new_im = build_image(new_im, [src_pyr[l]] ,[src_pyr[l]], patch_size=7)
#             else:
#                 new_im = build_image(new_im, [rsizer(src_pyr[l+1])] ,[src_pyr[l]], patch_size=7)
#         if l > 0:
#             rsizer = Resizer.Resizer(dst_pyr[l].shape, 4/3, dst_pyr[l-1].shape).cuda()
#             new_im = rsizer(new_im)
#     imshow(new_im)
#     print('Total time: %.2f[s]' % (time.time() - start,))
#     return new_im


# def summarization(pyr, ratio=(1/3,1/3), print_=False, patch_size=7, min_size=28, weighted_mean=False, gradual=True, eps=5e-3):
#     start = time.time()
#     new_im = pyr[0].clone()
#     imshow(new_im)
    
#     if gradual:
#         if ratio[0] <= 1 and ratio[1] <= 1:
#             num_steps = math.floor(max(math.log(ratio[0])/math.log(0.9), math.log(ratio[1])/math.log(0.9)))
#             step_size = [0.9, 0.9]
#             if ratio[0] > ratio[1]:
#                 step_size[0] = 10**(math.log10(ratio[0])/num_steps)
#             elif ratio[1] > ratio[0]:
#                 step_size[1] = 10**(math.log10(ratio[1])/num_steps)

#         elif ratio[0] >= 1 and ratio[1] >= 1:
#             num_steps = math.floor(max(math.log(ratio[0])/math.log(1.2), math.log(ratio[1])/math.log(1.2)))
#             step_size = [1.2, 1.2]
#             if ratio[0] < ratio[1]:
#                 step_size[0] = 10**(math.log10(ratio[0])/num_steps)
#             elif ratio[1] < ratio[0]:
#                 step_size[1] = 10**(math.log10(ratio[1])/num_steps)
                
#         elif ratio[0] < 1 and ratio[1] >= 1:
#             num_steps = math.floor(max(math.log(ratio[0])/math.log(0.9), math.log(ratio[1])/math.log(1.2)))
#             step_size = [1.2, 1.2]
#             if ratio[0] < ratio[1]:
#                 step_size[0] = 10**(math.log10(ratio[0])/num_steps)
#             elif ratio[1] < ratio[0]:
#                 step_size[1] = 10**(math.log10(ratio[1])/num_steps)
        
#         elif ratio[0] >= 1 and ratio[1] < 1:
#             num_steps = math.floor(max(math.log(ratio[0])/math.log(1.2), math.log(ratio[1])/math.log(0.9)))
#             step_size = [1.2, 1.2]
#             step_size[0] = 10**(math.log10(ratio[0])/num_steps)
#             step_size[1] = 10**(math.log10(ratio[1])/num_steps)

#         print(step_size, num_steps)
    
#     else:
#         step_size = [ratio[0],ratio[1]]
#         num_steps=1
    
#     results = []
#     for i in range(num_steps):
#         rsizer = Resizer.Resizer(new_im.shape, step_size).cuda()
#         new_im = rsizer(new_im)
#         new_pyr = get_pyramids_aux(new_im, 16, 0.8, False)
#         top_level = math.floor(min(math.log(min_size/new_im.shape[-2])/math.log(0.8), math.log(min_size/new_im.shape[-1])/math.log(0.8)))
        
#         print(top_level, new_pyr[top_level].shape, pyr[top_level].shape)
        
#         new_im = new_pyr[top_level].clone() #
#         start = time.time()
#         for l in range(top_level,-1,-1): 
#             print(l, new_pyr[l].shape, pyr[l].shape)
#             num_iters = 10
#             for k in range(num_iters):
#                 rsizer = Resizer.Resizer(pyr[l+1].shape, 5/4, pyr[l].shape).cuda()
#                 if weighted_mean:
#                     new_im = build_image_bidirectional_weighted(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], patch_size=7, eps=eps)
#                 else:
#                     new_im = build_image_bidirectional(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], patch_size=7, eps=eps)
#             imshow(new_im)
#             if l > 0:
#                 rsizer = Resizer.Resizer(new_pyr[l].shape, 5/4, new_pyr[l-1].shape).cuda()
#                 new_im = rsizer(new_im)
#         print('Scale ' + str(i) + ', Total time: %.2f[s]' % (time.time() - start,))
#         imshow(new_im)
#         results.append(new_im)
#     return new_im, results



def Montage(pyrs, new_im, num_steps=10, print_=False, patch_size=7, min_size=28, eps=5e-3, noise_std=0.75):
    start = time.time()
#     for i, pyr in enumerate(pyrs):
#         if i == 0:
#             new_im = pyr[0]
#         else:
#             new_im = torch.cat([new_im, pyr[0]], dim=-1)
#     imshow(new_im)
    
    step_size = [1, 0.9]
    
    for i in range(num_steps):
        rsizer = Resizer.Resizer(new_im.shape, step_size).cuda()
        new_im = rsizer(new_im)
        new_pyr = get_pyramids_aux(new_im, 15, 0.8, False)

        top_level = math.floor(min(math.log(min_size/new_im.shape[-2])/math.log(0.8), math.log(min_size/new_im.shape[-1])/math.log(0.8)))
        
        print(top_level, new_pyr[top_level].shape)
        
        new_im = new_pyr[top_level].clone() + new_pyr[top_level].clone()*noise_std
        start = time.time()
        for l in range(top_level,-1,-1):            
            num_iters = 10
            for k in range(num_iters):
                keys = []
                values = []
                for pyr in pyrs:
                    rsizer = Resizer.Resizer(pyr[l+1].shape, 5/4, pyr[l].shape).cuda()
                    keys.append(rsizer(pyr[l+1]))
                    keys.append(pyr[l])
                    values.append(pyr[l])
                    values.append(pyr[l])
                new_im = build_image_bidirectional(new_im, keys ,values, patch_size=7, eps=eps)
            imshow(new_im)
            if l > 0:
                rsizer = Resizer.Resizer(new_pyr[l].shape, 5/4, new_pyr[l-1].shape).cuda()
                new_im = rsizer(new_im)
        print('Shape: ' + str(new_im.shape) + ', Scale: ' + str(i) + ', Total time: %.2f[s]' % (time.time() - start,))
        imshow(new_im)

        
def structural_analogy(path_a, path_b, patch_size=7, top_layer=8, eps=5e-3, print_=False):
    start = time.time()
    pyr = get_pyramid(path_a,  None, 0, 20, 0.8, print_=print_) # Summary
    pyr2 = get_pyramid(path_b, None, 0, 20, 0.8, print_=print_) # Summary
    new_im = pyr2[0].clone()
    print(new_im.shape)
    for i in range(1):
        new_pyr = get_pyramids_aux(new_im, 20, 0.8, print_)
        new_im = new_pyr[top_layer].clone()
        for l in range(top_layer,-1,-1): 
            num_iters = 10
            for k in range(num_iters):
                rsizer = Resizer.Resizer(pyr[l+1].shape, 5/4, pyr[l].shape).cuda()
                new_im = build_image_bidirectional(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], patch_size=patch_size, eps=eps)
#             imshow(torch.cat([new_im, pyr2[l]], dim=-1))
            if l > 0:
                rsizer = Resizer.Resizer(new_pyr[l].shape, 5/4, new_pyr[l-1].shape).cuda()
                new_im = rsizer(new_im)
    print('Total time: %.2f[s]' % (time.time() - start,))
    imshow(torch.cat([new_im, pyr2[l]], dim=-1))
    return new_im


def conditional_inpainting(im, mask, patch_size=7, top_layer=8, print_=False):
    start = time.time()
    pyr = get_pyramids_aux(im, 15, 0.75, print_=print_) # Summary
    mask_pyr = get_pyramids_aux(mask, 15, 0.75, print_=print_) # Summary
    new_im = pyr[top_layer]
    for l in range(top_layer,-1,-1): 
        num_iters = 10
        for k in range(num_iters):
            rsizer = Resizer.Resizer(pyr[l+1].shape, 4/3, pyr[l].shape).cuda()
            new_im = build_image_mask(new_im, [rsizer(pyr[l+1]), pyr[l]] ,[pyr[l], pyr[l]], [rsizer(mask_pyr[l+1]), mask_pyr[l]], patch_size=patch_size)
        imshow(new_im)
        if l > 0:
            rsizer = Resizer.Resizer(pyr[l].shape, 4/3, pyr[l-1].shape).cuda()
            new_im = rsizer(new_im)

    print('Total time: %.2f[s]' % (time.time() - start,))
    imshow(new_im)
    imshow(pyr[0])
    return new_im




Files already downloaded and verified
Files already downloaded and verified
tensor([5])
Total time: 5.03[s]
Total time: 2.94[s]
Total time: 2.84[s]
Total time: 2.94[s]
Total time: 2.91[s]
Total time: 2.88[s]
Total time: 2.90[s]
Total time: 2.84[s]
Total time: 2.92[s]
Total time: 2.87[s]
Total time: 2.79[s]
Total time: 2.77[s]
Total time: 2.74[s]
Total time: 2.78[s]
Total time: 2.66[s]
Total time: 2.76[s]
Total time: 2.77[s]
Total time: 2.67[s]
Total time: 2.72[s]
Total time: 2.69[s]
Total time: 2.82[s]
Total time: 2.77[s]
Total time: 2.77[s]
Total time: 2.72[s]
Total time: 2.74[s]
Total time: 2.78[s]
Total time: 2.80[s]
Total time: 2.71[s]
Total time: 2.76[s]
Total time: 2.74[s]
Total time: 2.85[s]
Total time: 2.73[s]
Total time: 2.75[s]
Total time: 2.74[s]
Total time: 2.73[s]
Total time: 2.73[s]
Total time: 2.74[s]
Total time: 2.80[s]
Total time: 2.74[s]
Total time: 2.74[s]
Total time: 2.84[s]
Total time: 2.81[s]
Total time: 2.69[s]
Total time: 2.71[s]
Total time: 2.73[s]
Total time: 

KeyboardInterrupt: 

In [ ]:
files = os.listdir('/home/nivg/Projects/mturk-dg/exp1/sigd_dg/')

for file in files:
    pyr = get_pyramid('/home/nivg/Projects/mturk-dg/exp1/sigd_dg/' + file, None, 0, 1, 0.75, print_=False) 
    pyr_gt = get_pyramid('/home/nivg/Projects/mturk-dg/exp1/sigd_gt/' + file, None, 0, 1, 0.75, print_=False) 
    print(file, get_psnr(pyr[0],pyr_gt[0]))
    imshow((pyr[0] - pyr_gt[0]).abs())

In [ ]:
# Birds, mountains Comparison
pyr = get_pyramid('/home/nivg/data/mountains.jpg', None, 0, 15, 0.75, print_=True) 
pyr=pyr[3:]
for ratio in ([[1,1],[1,4/5],[1,3/4],[1,1/3],[1,5/4],[1,2]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        im = new_image_generation(pyr, dst_pyr)
#         imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Birds, mountains Comparison
pyr = get_pyramid('/home/nivg/data/DropGAN/user_study/newDS_jpg/birds_3.jpg', None, 0, 15, 0.75, print_=True) 
for ratio in ([[1,1]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(10):
        im = new_image_generation_adaptive(pyr, dst_pyr)
        imshow(im)
        print(get_psnr(pyr[0], im))
#         imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(r[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Animation attempt
images = []
BASE_PATH = '/home/nivg/data/'
files = os.listdir(BASE_PATH)
for file in ['wild_bush.png']:#,'smiley.jpg','crazy-building.jpg','checkers.jpg']:#['cookies.jpg', 'ladakh_horses.jpg', 'vietnam_traffic.jpg', 'people_on_the_beach.jpg','smiley.jpg',]:
    pyr = get_pyramid(BASE_PATH + file, None, 0, 15, 0.75, print_=False)
    im = pyr[0].detach().clone()
    for ratio in ([[1,1]]):
        for k in range(30):
#             pyr = get_pyramids_aux(im, num_layers=15, ratio=0.75, print_=False)
            dst_pyr = get_pyramids_aux(im, num_layers=15, ratio=0.75, print_=False)
            im = new_image_generation_adaptive(pyr, dst_pyr,top_level=2, noise_std=1.75)
            imshow(im)
            print(get_psnr(im, pyr[0]))
            images.append(im.detach().clone().squeeze(0).permute(1,2,0).cpu())
#             imwrite('/home/nivg/data/DropGAN/image_generation/' + file.split('.')[0] + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '_hr.png', im.squeeze(0).detach().cpu())

In [ ]:
# Mountains3 Comparison
for ratio in ([[1,1],[1,1/3],[1,5/4],[4/5,1],[1,2]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        im = new_image_generation(pyr, dst_pyr)
        imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Balloons Comparison
for ratio in ([[1,1],[1,1/3],[1,5/4],[5/4,1],[1,2]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        im = new_image_generation(pyr, dst_pyr)
        imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Balloons/Colusseum/Birds for figure
pyr = get_pyramid('/home/nivg/data/corn.png', None, 0, 15, 0.75, print_=True) 
pyr = pyr[1:]
for ratio in ([[1,1]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(10):
        im = new_image_generation(pyr, dst_pyr)
        imshow(im)
        print(get_psnr(im, pyr[0]))
#         imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# colusseum Comparison
pyr = get_pyramid('/home/nivg/data/colusseum.png', None, 0, 15, 0.75, print_=True) 
# pyr=pyr[3:]
imshow(pyr[0])
for ratio in ([[1,1],[1,4/3],[4/3,1]]):
    rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
    dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
    for k in range(2):
        if ratio == [4/3,1]:
            im = new_image_generation(pyr, dst_pyr, top_level=8)
        else:
            im = new_image_generation(pyr, dst_pyr)
#         imwrite('/home/nivg/data/DropGAN/image_generation/' + IM_NAME + '_' + str(ratio[0]) + '_' + str(ratio[1]) +  '_im' + str(k) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
im = harmonization(pyr, pyr_harm, top_level=0, patch_size=7)
# imwrite('/home/nivg/data/DropGAN/image_harmonization/' + IM_NAME + '_harmonized.png', im.squeeze(0).detach().cpu())

In [ ]:
# Tree Edit
IM_PATH = '/home/nivg/data/stone.png'
EDIT_IM_PATH = '/home/nivg/data/stone_edit.png'
IM_NAME = 'stone'
pyr = get_pyramid(IM_PATH, None, 0, 15, 0.75, print_=True)
pyr_edit = get_pyramid(EDIT_IM_PATH, None, 0, 15, 0.75, print_=True)

In [ ]:
# for i in range(10):
#     im = edit(pyr, pyr_edit, mask_size=44, patch_size=7, top_level_max=9)
#     imwrite('/home/nivg/data/DropGAN/image_editing/' + IM_NAME + '_edited_ver' + str(i) + '.png', im.squeeze(0).detach().cpu())

In [ ]:
# Stone Edit
IM_PATH = '/home/nivg/data/DropGAN/user_study/newDS_jpg/baseball.jpg'
EDIT_IM_PATH = '/home/nivg/data/DropGAN/image_editing/baseball_edited.jpg'
IM_NAME = 'stone'
# pyr = get_pyramid(IM_PATH, None, 0, 15, 0.75, print_=True) 
mask = torch.zeros_like(pyr_edit[0])
mask[:,:,50:110, 190:245] = 1
# mask[:,:,50:115, 140:180] = 1

imshow(mask)
# pyr_edit = get_pyramid(EDIT_IM_PATH, None, 0, 15, 0.75, print_=True)
# im = edit(pyr, pyr_edit, mask_size=35, patch_size=7, top_level_max=9)
# imwrite('/home/nivg/data/DropGAN/image_editing/' + IM_NAME + '_edited.png', im.squeeze(0).detach().cpu())

In [ ]:
# Structural Analogies \ Paint to Image

# 206, 207 - 10, 5e-3
# 207, 206 - 8, 5e-2
# 204, 205 - 4, 5e-1
# 205, 204 - 4, 5e-3
# 202, 203 - 4, 5e-4
# 203, 202 - 2, 5e-4
# 208_resized, 209 - 10, 5e-3
# 209, 208_resized - 6, 5e-3
# 10, 11 - 6, 5e-3
# 108, 109 - 4, 5e-3
# 109, 108 - 6, 5e-3
# trees_paint, trees - 3, 5e-2
# cows_paint, cows - 3, 5e-4
# 309_resized, 308 - 6, 5e-3

# 4_real_a, 4_real_b - 6, 5e-3
# 4_real_b, 4_real_a - 6, 5e-2

# 10_real_a, 10_real_b - 6, 5e-3
# 10_real_b, 10_real_a - 7, 5e-3

# factory_real_a, factory_real_b - 8, 5e-2
# factory_real_b, factory_real_a - 8, 5e-3

# flowers_real_a, flowers_real_b - 7, 5e-3
# flowers_real_b, flowers_real_a - 7, 5e-3

# flowers2_real_b, flowers2_real_a - 11, 5e-3
# flowers2_real_a, flowers2_real_b - 10, 5e-3

# knit_real_a, knit_real_b - 3/4, 5e-4
# knit_real_b, knit_real_a - 6, 5e-3

# perspective_real_a, perspective_real_b - 4, 5e-3
# perspective_real_b, perspective_real_a - 8, 5e-2

# snow_real_a (1), snow_real_b (2) - 5, 5e-3
# snow_real_b (2), snow_real_a (1) - 4, 5e-2

# marble_real_a, marble_real_b - 8, 5e-3
# marble_real_b, marble_real_a - 8, 5e-3

# 0_real_a, 0_real_b - 8, 5e-4
# 0_real_b, 0_real_a - 8, 5e-4

# sky_real_a, sky_real_b - 6, 5e-4
# sky_real_b, sky_real_a - 6, 5e-4

# oranges_real_a, oranges_real_b - 6, 5e-4
# oranges_real_b, oranges_real_a - 7, 5e-4

# tennis_real_a (1), tennis_real_b - 6, 5e-5
# tennis_real_b, tennis_real_a (1) - 9, 5e-3

# dirtroads_real_a, dirtroads_real_b - 7, 5e-3
# dirtroads_real_b, dirtroads_real_a - 8, 5e-4

# snow2ice_real_a, snow2ice_real_b - 10, 5e-3
# snow2ice_real_b, snow2ice_real_a - 11, 5e-2

# 22_real_a, 22_real_b - 8, 5e-4 / 10,5e-4
# 22_real_b, 22_real_a - 7, 5e-4

#### tui
# tui_B, tui_A - 1, 5e-4 / 11, 5e-5
# tui2_A, tui2_B - 10, 5e-2
# tui2_B, tui2_A - 9, 5e-2 / 8, 5e-5
images = []
for i in range(1,26):
    IM_PATH_B = '/home/nivg/data/NotVGAN/frames/mnist_morph/' + str(i+24) + '.png'
    IM_PATH_A = '/home/nivg/data/NotVGAN/frames/soldiers/' + str(i) + '.png'
    # for l in range(7,9):
    #     for eps in ([5e0, 5e-1, 5e-2, 5e-3, 5e-4, 5e-5, 5e-6]):
    #         print(l, eps)
    # im = structural_analogy(IM_PATH_A, IM_PATH_B, patch_size=2, top_layer=0, eps=5e-3, print_=False)
    im = structural_analogy_nivh(IM_PATH_A, IM_PATH_B, patch_size=7, top_layer=7, eps=1e-3, print_=False)
    images.append(im)
# imshow(imread(IM_PATH_B))


In [85]:
# plowing - 21
# fruit - 28
# corn - 21 
# rome - 35
# knitting - 35
# farm house - 35 (large - from pyr[1:])
# capitol - 21 (large - from pyr[1:])
# birds - 21
# man - 8
# cab_building - 35
# cake - 28
# pantheon - 35
# houses - 35
# particle accelerator - 21
# ziskind2 - 49
# ziskind - 35
# shirts - 21
# lattice - 21
# crazy-building - 42

###### single step (for image expansion):
# farm_house - 49 (from pyr[1:])
# corn - 11 (from pyr[1:])
# rome - 35 (from pyr[2:])
# fruit - 21 (from pyr[0:])
# knitting - 21 (from pyr[0:])
# man - 7 (from pyr[0:])
# plowing - 21 (from pyr[0:])
# birds - 11 (from pyr[1:])
# cab_building - 11 (from pyr[1:])
# cake - 35 (from pyr[5:])
# houses - 35 (from pyr[3:])
# pantheon - 21 (from pyr[3:])
# capitol - 14 (from pyr[2:])

SUMMARY_IM_PATH = '/home/nivg/data/DropGAN/Analogies/for_assaf/marble_real_a.jpg'
pyr = get_pyramid(SUMMARY_IM_PATH, None, 0, 20, 0.8, print_=False) 
# pyr = pyr[3:]
print(pyr[0].shape)
for i in range(1):
    im, results = summarization(pyr, ratio=(1,0.5), print_=True, min_size=21, weighted_mean=True, gradual=False, eps=5e-1)


torch.Size([1, 3, 220, 220])


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xdc\x00\x00\x00\xdc\x08\x02\x00\x00\x00\x948X\xd…

7 torch.Size([1, 3, 47, 24]) torch.Size([1, 3, 47, 47])
7 torch.Size([1, 3, 47, 24]) torch.Size([1, 3, 47, 47])


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x18\x00\x00\x00/\x08\x02\x00\x00\x00vj\x9d\xba\x…

6 torch.Size([1, 3, 58, 29]) torch.Size([1, 3, 58, 58])


...

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00X\x00\x00\x00\xb0\x08\x02\x00\x00\x00\x04\xe0>\x8…

0 torch.Size([1, 3, 220, 110]) torch.Size([1, 3, 220, 220])


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00n\x00\x00\x00\xdc\x08\x02\x00\x00\x00\x1e;\x03\xd…

Scale 0, Total time: 3.72[s]


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00n\x00\x00\x00\xdc\x08\x02\x00\x00\x00\x1e;\x03\xd…

In [ ]:
for i,im in enumerate(results):
    ratio_h = 1**(i+1)
    ratio_w = 0.9**(i+1)
    save_dir = '/home/nivg/data/DropGAN/Retargeting/sunflower_%.2f_%.2f' % (ratio_h, ratio_w)
    save_dir = save_dir.split('.')[0] + '_' + save_dir.split('.')[1] + '_' + save_dir.split('.')[2] + '.png'
    print(save_dir)
    imwrite(save_dir, im.squeeze(0))


In [ ]:
from statistics import NormalDist

NormalDist(mu=0, sigma=0.85).cdf(0.02) 

In [ ]:
# Montage gardens + girl
pyr1 = get_pyramid('/home/nivg/data/Tapestry_input_1.png', None, 0, 15, 0.8, print_=True) # Summary
pyr2 = get_pyramid('/home/nivg/data/Tapestry_input_2.png', None, 0, 15, 0.8, print_=True) # Summary
pyr3 = get_pyramid('/home/nivg/data/Tapestry_input_3.png', None, 0, 15, 0.8, print_=True) # Summary
m0 = torch.nn.ConstantPad2d((0,0,32,0),0.5)
m1 = torch.nn.ConstantPad2d((0,0,32,0),0.66)
m2 = torch.nn.ConstantPad2d((0,0,32,0),0.8)
new_im = torch.cat([pyr1[0], pyr2[0], torch.cat([m0(pyr3[0][:,0,:,:]), m1(pyr3[0][:,1,:,:]), m2(pyr3[0][:,2,:,:])]).unsqueeze(0)], dim=-1)
imshow(new_im)

Montage([pyr1, pyr2, pyr3], new_im, num_steps=9, print_=False, patch_size=7, min_size=8)# with noise
# Montage([pyr1, pyr2, pyr3], new_im, num_steps=9, print_=False, patch_size=7, min_size=21)# without noise

In [ ]:
# Montage iceland
pyr1 = get_pyramid('/home/nivg/data/iceland_1.png', None, 0, 15, 0.8, print_=True) # Summary
pyr2 = get_pyramid('/home/nivg/data/iceland_2.png', None, 0, 15, 0.8, print_=True) # Summary
pyr3 = get_pyramid('/home/nivg/data/iceland_3.png', None, 0, 15, 0.8, print_=True) # Summary
pyr1 = pyr1[3:]
pyr2 = pyr2[3:]
pyr3 = pyr3[3:]
# imshow(new_im)
new_im = torch.cat([pyr1[0], pyr2[0], pyr3[0]], dim=-1)
Montage([pyr1, pyr2, pyr3], new_im, num_steps=9, print_=False, patch_size=7, min_size=35)

In [ ]:
pyr1 = get_pyramid('/home/nivg/data/DropGAN/Collage/man.jpg', None, 0, 15, 0.8, print_=True) # Summary
pyr2 = get_pyramid('/home/nivg/data/DropGAN/Collage/woman.jpg', None, 0, 15, 0.8, print_=True) # Summary
# pyr1 = pyr1[3:]
# pyr2 = pyr2[3:]
# pyr3 = pyr3[3:]
# imshow(new_im)

new_im = torch.cat([pyr1[0], pyr2[0]], dim=-1)
Montage([pyr1,pyr2], new_im, num_steps=9, print_=False, patch_size=7, min_size=7, eps=5e-3)

In [ ]:
pyr1 = get_pyramid('/home/nivg/data/DropGAN/Collage/rain_1.jpg', None, 0, 16, 0.8, print_=True) # Summary
pyr2 = get_pyramid('/home/nivg/data/DropGAN/Collage/rain_2.jpg', None, 0, 16, 0.8, print_=True) # Summary
# pyr3 = get_pyramid('/home/nivg/data/DropGAN/Collage/children3.jpg', None, 0, 16, 0.8, print_=True) # Summary

# pyr2 = pyr2[1:]
# pyr2 = pyr2[3:]
# pyr3 = pyr3[3:]
# imshow(new_im)
m0 = torch.nn.ConstantPad2d((0,0,44,0),0)
m1 = torch.nn.ConstantPad2d((0,0,44,0),0)
m2 = torch.nn.ConstantPad2d((0,0,44,0),1)

new_im = torch.cat([pyr2[0], pyr1[0]], dim=-1)
# new_im = torch.cat([pyr1[0], torch.cat([m0(pyr2[0][:,0,:,:]), m1(pyr2[0][:,1,:,:]), m2(pyr2[0][:,2,:,:])]).unsqueeze(0)], dim=-1)
imshow(new_im)
Montage([pyr3,pyr1,pyr2], new_im, num_steps=9, print_=False, patch_size=7, min_size=21, eps=5e-3, noise_std=0.0)

In [ ]:
pyr1 = get_pyramid('/home/nivg/data/DropGAN/Collage/f1.png', None, 0, 15, 0.8, print_=True) # Summary
pyr2 = get_pyramid('/home/nivg/data/DropGAN/Collage/f2.png', None, 0, 15, 0.8, print_=True) # Summary
pyr3 = get_pyramid('/home/nivg/data/DropGAN/Collage/f3.png', None, 0, 15, 0.8, print_=True) # Summary

pyr1 = pyr1[1:]
pyr2 = pyr2[1:]
pyr3 = pyr3[1:]

new_im = torch.cat([pyr1[0], pyr2[0], pyr3[0]], dim=-1)
imshow(new_im)
Montage([pyr1, pyr2, pyr3], new_im, num_steps=9, print_=False, patch_size=7, min_size=7)

In [ ]:
# # Conditional Inpainting
# pyr = get_pyramid('/home/nivg/data/204.jpg', None, 0, 15,0.75)
# mask = torch.ones_like(pyr[0])
# mask[:,:,50:75, 20:40] = 0
# new_im = pyr[0].clone()
# for i in range(3):
#     if i == 2:
#         new_im[:,i,50:75, 20:40] = 0
#     else:
#         new_im[:,i,50:75, 20:40] = 1
# conditional_inpainting(new_im, mask, patch_size=7, top_layer=5, print_=False)

In [ ]:
# Conditional Inpainting for figure
pyr = get_pyramid('/home/nivg/data/BSDS100/296007.png', None, 0, 15,0.75)
pyr = pyr[2:]
mask = torch.ones_like(pyr[0])
# mask[:,:,140:165, 160:190] = 0
mask[:,:,80:120, 10:120] = 0

new_im = pyr[0].clone()#*mask
imshow(new_im)
for i in range(1):
    if i == 0:
#         new_im[:,:,140:165, 160:190] = 1.0
        
#         for j in range(3):
#             if j == 1:
#                 new_im[:,j,140:165, 160:190] = 0.7
#             else:
#                 new_im[:,j,140:165, 160:190] = 0
        
        for j in range(3):
            if j == 1:
                new_im[:,j,80:120, 10:120] = 0.7
            else:
                new_im[:,j,80:120, 10:120] = 0

        
    else:
        for j in range(3):
            if j < 2:
                new_im[:,j,140:165, 160:190] = 0.7
            else:
                new_im[:,j,140:165, 160:190] = 0.3
    conditional_inpainting(new_im, mask, patch_size=7, top_layer=6, print_=False)

In [ ]:
# Conditional Inpainting for figure
pyr = get_pyramid('/home/nivg/data/DropGAN/potential_new_inputs/checkers.jpg', None, 0, 15,0.75, False)
mask = torch.ones_like(pyr[0])
mask[:,:,120:135, 50:70] = 0
new_im = pyr[0].clone()*mask
imshow(new_im)
for i in range(2):
    if i == 0:
        new_im[:,:,120:135, 50:70] = 0.0
    else:
        new_im[:,:,120:135, 50:70] = 1
    conditional_inpainting(new_im, mask, patch_size=7, top_layer=6, print_=False)

In [ ]:
# Conditional Inpainting for figure
pyr = get_pyramid('/home/nivg/data/DropGAN/Analogies/for_assaf/marble_real_a.jpg', None, 0, 15,0.75, False)
mask = torch.ones_like(pyr[0])
mask[:,:,170:220, 10:50] = 0
new_im = pyr[0].clone()*mask
imshow(new_im)
for i in range(3):
    if i == 0:
        for j in range(3):
            if j == 2:
                new_im[:,j,170:220, 10:50] = 1
            else:
                new_im[:,j,170:220, 10:50] = 0
    elif i == 1:
        for j in range(3):
            if j == 1:
                new_im[:,j,170:220, 10:50] = 1
            else:
                new_im[:,j,170:220, 10:50] = 0
    else:
        for j in range(3):
            if j == 2:
                new_im[:,j,170:220, 10:50] = 0.9
            elif j == 1:
                new_im[:,j,170:220, 10:50] = 0.5
            else:
                new_im[:,j,170:220, 10:50] = 0

    new_im = conditional_inpainting(new_im, mask, patch_size=7, top_layer=6, print_=False)


In [ ]:
# Conditional Inpainting for figure
pyr = get_pyramid('/home/nivg/data/fruit.png', None, 0, 15,0.75, False)
mask = torch.ones_like(pyr[0])
mask[:,:,105:120, 125:165] = 0
new_im = pyr[0].clone()*mask
imshow(new_im)
for i in range(3):
    if i == 0:
        for j in range(3):
            if j == 0:
                new_im[:,j,105:120, 125:165] = 1
            else:
                new_im[:,j,105:120, 125:165] = 0
    elif i == 1:
        for j in range(3):
            if j == 2:
                new_im[:,j,105:120, 125:165] = 0
            else:
                new_im[:,j,105:120, 125:165] = 1
    else:
        for j in range(3):
            if j == 2:
                new_im[:,j,105:120, 125:165] = 0.9
            elif j == 1:
                new_im[:,j,105:120, 125:165] = 0.
            else:
                new_im[:,j,105:120, 125:165] = 0

    conditional_inpainting(new_im, mask, patch_size=7, top_layer=3, print_=False)
#     new_im = conditional_inpainting(new_im, mask, patch_size=7, top_layer=4, print_=False)

In [ ]:
# Conditional Inpainting for figure
pyr = get_pyramid('/home/nivg/data/DropGAN/Conditional_inpainting/library.png', None, 0, 15,0.75, False)

mask = get_pyramid('/home/nivg/data/DropGAN/Conditional_inpainting/library_mask.png', None, 0, 15,0.75, False)
mask = mask[3]
imshow(mask)
# mask[:,:,105:120, 125:165] = 0
new_im = pyr[3].clone()*mask
new_im[(1-mask).byte()] = torch.randn_like(pyr[3])[(1-mask).byte()] + 0.5
print(new_im.shape)
imshow(new_im)
for i in range(1):
#     if i == 0:
#         for j in range(3):
#             if j == 0:
#                 new_im[:,j,105:120, 125:165] = 1
#             else:
#                 new_im[:,j,105:120, 125:165] = 0
#     elif i == 1:
#         for j in range(3):
#             if j == 2:
#                 new_im[:,j,105:120, 125:165] = 0
#             else:
#                 new_im[:,j,105:120, 125:165] = 1
#     else:
#         for j in range(3):
#             if j == 2:
#                 new_im[:,j,105:120, 125:165] = 0.9
#             elif j == 1:
#                 new_im[:,j,105:120, 125:165] = 0.
#             else:
#                 new_im[:,j,105:120, 125:165] = 0

#     conditional_inpainting(new_im, mask, patch_size=7, top_layer=6, print_=False)
    new_im = conditional_inpainting(new_im, mask, patch_size=4, top_layer=5, print_=False)


In [ ]:
##### figure 4 (PNN) 
orig_pyr = get_pyramid('/home/nivg/data/balloons.png', None, 0, 15,0.75)
pyr = get_pyramid('/home/nivg/data/DropGAN/balloons_for_figure.png', None, 0, 15,0.75)

unfold = torch.nn.Unfold(kernel_size=(29,29))
imshow(pyr[3])
rsizer = Resizer.Resizer(pyr[5].shape, (4/3)**2, pyr[3].shape).cuda()
imshow(rsizer(pyr[5]))
imshow(rsizer(orig_pyr[5]))
imshow(orig_pyr[3])
q_patches = unfold(rsizer(pyr[5]))
k_patches = unfold(rsizer(orig_pyr[5]))
v_patches = unfold(orig_pyr[3])
print(in_patches.shape[-1])
for i in range(q_patches.shape[-1]):
    if i % 10 == 1:
        print(i)
        imshow(q_patches[:,:,i].reshape(1,3,29,29))
        imshow(k_patches[:,:,i].reshape(1,3,29,29))
        imshow(v_patches[:,:,i].reshape(1,3,29,29))


In [ ]:
# SIFID


SIFID_REAL_DIR = '/home/nivg/data/DropGAN/user_study/real/'
files = os.listdir(SIFID_REAL_DIR)
for file in files:
    if file.split('.')[1] == 'jpg':
        print(file)
        pyr = get_pyramid(SIFID_REAL_DIR + file, None, 0, 15, 0.75, print_=False) 
        for ratio in ([[1,1]]):
            rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
            dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
            for k in range(5):
                im = new_image_generation(pyr, dst_pyr)
                imshow(torch.cat([im, pyr[0], np2pt(imread('/home/nivg/data/DropGAN/user_study/fake_mid_variance/' + file)).cuda()], dim=-1))
                print(get_psnr(im, pyr[0]))
#                 imwrite('/home/nivg/data/DropGAN/user_study/fake_DropGAN_std_1_25/' + file.split('.')[0] + '.png', im.squeeze(0).detach().cpu())


In [ ]:
# SIFID

SIFID_REAL_DIR = '/home/nivg/data/DropGAN/user_study/real/'
files = os.listdir(SIFID_REAL_DIR)
count = 0
for file in files:
    count += 1
    print(count)
    if file.split('.')[1] == 'jpg':
        pyr = get_pyramid(SIFID_REAL_DIR + file, None, 0, 15, 0.75, print_=False) 
        for ratio in ([[1,1]]):
            rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
            dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
            for k in range(50):
                print(k)
                im = new_image_generation(pyr, dst_pyr)
                print(get_psnr(im, pyr[0]))
                imwrite('/home/nivg/data/DropGAN/user_study/fake_DropGAN_variability_std_1_25_test_png/' + 'repeat_' + str(k) + '_' + file.split('.')[0] + '.png', im.squeeze(0).detach().cpu())


In [ ]:
# SIFID new DS

SIFID_REAL_DIR = '/home/nivg/data/DropGAN/user_study/newDS_jpg/'
files = os.listdir(SIFID_REAL_DIR)
count = 0
for file in files:
    count += 1
    print(count)
    if file in ['vietnam_traffic.jpg']:
        pyr = get_pyramid(SIFID_REAL_DIR + file, None, 0, 15, 0.75, print_=False) 
        for ratio in ([[1,1]]):
            rsizer = Resizer.Resizer(pyr[0].shape, ratio).cuda()
            dst_pyr = get_pyramids_aux(rsizer(pyr[0]), num_layers=15, ratio=3/4, print_=False)
            for k in range(50):
                print(k)
                im = new_image_generation(pyr, dst_pyr)
                print(get_psnr(im, pyr[0]))
                imwrite('/home/nivg/data/DropGAN/user_study/fake_DropGAN_variability_std_0_75_new_DS/' + file.split('.')[0] + '/' +  str(k) + '.png', im.squeeze(0).detach().cpu())


In [ ]:
a = torch.randn(5,10)
ind = torch.randint(low=0, high=10, size=(5,))
print(a)
print(ind)

In [ ]:
# for unconditional figure
BASE_PATH = '/home/nivg/data/'
pyr = get_pyramid(BASE_PATH + 'carrots_s.png', None, 0, 15, 0.75, print_=False)
pyr=pyr[1:]
images = []
noisy = []
for i in range(50):
    n = torch.randn_like(pyr[9])*0.75
    new_im = build_image_weighted(pyr[9] + n, [pyr[9]] ,[pyr[9]], patch_size=7)
    imshow(pyr[9]+n)
#     print(new_im.shape)
    noises.append(n) 
#     images.append(new_im) 
    new_im *= 255
    images.append(new_im.detach().clone().squeeze(0).permute(1,2,0).cpu().type(torch.uint8))
    
#     noise = (n.clone() + 1)*255/2
    noisy.append((255*(pyr[9] + n)).detach().clone().squeeze(0).permute(1,2,0).cpu().type(torch.uint8))


In [ ]:
kwargs_write = {'fps':4.0, 'quantizer':'nq'}
imageio.mimsave('/home/nivg/data/YouTube/noise.gif',noisy[1::2],'GIF-FI', **kwargs_write)
# print(noises[49])

In [ ]:
import imageio
imageio.plugins.freeimage.download()
kwargs_write = {'fps':1.0, 'quantizer':'nq'}

folder = '/home/nivg/data/DropGAN/user_study/fake_DropGAN_variability_std_0_75_new_DS/balloons/'
gt = np.expand_dims(imread('/home/nivg/data/DropGAN/user_study/newDS_jpg/people_on_the_beach.jpg'),0)
gt = torch.tensor(gt).contiguous().permute(0,3,1,2).detach().cuda()
images = []
for file in os.listdir(folder):
    im = np.expand_dims(imread(folder + file),0)
    im = torch.tensor(im).contiguous().permute(0,3,1,2).detach().cuda()
    im *= 255
#     if get_psnr(im,gt) <= 35:
    images.append(im.detach().clone().squeeze(0).permute(1,2,0).cpu().type(torch.uint8))
imageio.mimsave('/home/nivg/data/DropGAN/balloons.gif',images,'GIF-FI', **kwargs_write)


In [ ]:
import imageio
folder = '/home/nivg/data/DropGAN/user_study/fake_DropGAN_variability_std_1_25_test_png/'
gt = np.expand_dims(imread('/home/nivg/data/DropGAN/user_study/real/8.jpg'),0)
gt = torch.tensor(gt).contiguous().permute(0,3,1,2).detach().cuda()
images = []
for file in os.listdir(folder):
    if file.endswith('_8.png'): 
        im = np.expand_dims(imread(folder + file),0)
        im = torch.tensor(im).contiguous().permute(0,3,1,2).detach().cuda()
#         if get_psnr(im,gt) <= 35:
        images.append(im.detach().clone().squeeze(0).permute(1,2,0).cpu())
imageio.mimsave('/home/nivg/data/DropGAN/8.gif',images,'GIF-FI', **kwargs_write)


In [ ]:
import imageio
for i in ['8']:
    folder = '/home/nivg/Projects/SinGAN/Output/RandomSamples/' + str(i) + '/gen_start_scale=0/'
#     gt = np.expand_dims(imread('/home/nivg/data/DropGAN/user_study/newDS_jpg/people_on_the_beach.jpg'),0)
#     gt = torch.tensor(gt).contiguous().permute(0,3,1,2).detach().cuda()
    images = []
    for file in os.listdir(folder):
        im = np.expand_dims(imread(folder + file),0)
        im = torch.tensor(im).contiguous().permute(0,3,1,2).detach().cuda()
#         if get_psnr(im,gt) <= 35:
        images.append(im.detach().clone().squeeze(0).permute(1,2,0).cpu())
    imageio.mimsave('/home/nivg/data/DropGAN/' + str(i) + '_singan.gif',images,'GIF-FI', **kwargs_write)


In [ ]:
import imageio
folder = '/home/nivg/data/DropGAN/user_study/newDS_jpg/'
sg_folder = '/home/nivg/data/DropGAN/user_study/fake_SinGAN_newDS_level_N/RandomSamples/'
dg_folder = '/home/nivg/data/DropGAN/user_study/fake_DropGAN_variability_std_0_75_new_DS/'
kwargs_write = {'fps':1.0, 'quantizer':'nq'}

for file in os.listdir(folder):
#     gt = np.expand_dims(imread('/home/nivg/data/DropGAN/user_study/newDS_jpg/' + file),0)
#     gt = torch.tensor(gt).contiguous().permute(0,3,1,2).detach().cuda()
    sg_images = []
    dg_images = []
    for l in range(50):
        sg_im = np.expand_dims(imread(sg_folder + file.split('.')[0] + '/' + 'gen_start_scale=0/' + str(l) + '.png'),0)
        sg_im = torch.tensor(sg_im).contiguous().permute(0,3,1,2).detach().cuda()
        sg_im *= 255
        sg_images.append(sg_im.detach().clone().squeeze(0).permute(1,2,0).cpu().type(torch.uint8))
        dg_im = np.expand_dims(imread(dg_folder + file.split('.')[0] + '/' + str(l) + '.png'),0)
        dg_im = torch.tensor(dg_im).contiguous().permute(0,3,1,2).detach().cuda()
        dg_im *= 255
        dg_images.append(dg_im.detach().clone().squeeze(0).permute(1,2,0).cpu().type(torch.uint8))

#         if get_psnr(im,gt) <= 35:
#             images.append(im.detach().clone().squeeze(0).permute(1,2,0).cpu())
    imageio.mimsave('/home/nivg/data/DropGAN/supp/' + file.split('.')[0] + '_sg.gif', sg_images, 'GIF-FI', **kwargs_write)
    imageio.mimsave('/home/nivg/data/DropGAN/supp/' + file.split('.')[0] + '_dg.gif',dg_images,'GIF-FI', **kwargs_write)

In [ ]:
# f = open("/home/nivg/data/DropGAN/template.txt", "r")
# txt = f.read()

# folder = '/home/nivg/data/DropGAN/user_study/newDS_jpg/'
# new_txt = ""
# new_file = ""
# for file in os.listdir(folder):
#     new_txt = txt.replace("method_010", "method_" + file.split('.')[0])
#     new_txt = new_txt.replace("010_rand_ker", file.split('.')[0])
#     new_txt = new_txt.replace("bicubic", "source")
#     new_txt = new_txt.replace("edsr", "GPNN")
#     new_txt = new_txt.replace("michaeli", "SinGAN")
#     new_txt = new_txt.replace("psnr_007", "measures_" + file.split('.')[0])
#     new_txt = new_txt.replace("psnr_010", "measures_" + file.split('.')[0])
#     new_txt = new_txt.replace("birds_3.jpg", file)
# #     print(new_txt)
#     new_file = new_file + new_txt + '\n'
# print(new_file)

f = open("/home/nivg/data/DropGAN/template2.txt", "r")
txt = f.read()

folder = '/home/nivg/data/DropGAN/user_study/newDS_jpg/'
new_txt = ""
new_file = ""
files = os.listdir(folder)
for i in range(0,len(files),2):
    file = files[i]
    print(file)
    new_txt = txt.replace("method_dolphins", "method_" + file.split('.')[0])
    new_txt = new_txt.replace("dolphins", file.split('.')[0])
    new_txt = new_txt.replace("measures_dolphins", "measures_" + file.split('.')[0])
    new_txt = new_txt.replace("measures_dolphins", "measures_" + file.split('.')[0])
    new_txt = new_txt.replace("dolphins.jpg", file)
    file = files[i+1]
    print(file)
    new_txt = new_txt.replace("method_208", "method_" + file.split('.')[0])
    new_txt = new_txt.replace("208", file.split('.')[0])
    new_txt = new_txt.replace("measures_208", "measures_" + file.split('.')[0])
    new_txt = new_txt.replace("measures_208", "measures_" + file.split('.')[0])
    new_txt = new_txt.replace("208.jpg", file)

#     print(new_txt)
    new_file = new_file + new_txt + '\n'
# print(new_file)

In [ ]:
# files = """3.jpg
# 18.jpg
# 44.jpg
# 8.jpg
# 34.jpg
# 25.jpg
# 10.jpg
# 35.jpg
# 36.jpg
# 30.jpg
# 48.jpg
# 11.jpg
# 12.jpg
# 43.jpg
# 21.jpg
# 13.jpg
# 1.jpg
# 39.jpg
# 2.jpg
# 5.jpg
# 33.jpg
# 46.jpg
# 50.jpg
# 19.jpg
# 40.jpg
# 47.jpg
# 49.jpg
# 16.jpg
# 31.jpg
# 9.jpg
# 26.jpg
# 6.jpg
# 37.jpg
# 42.jpg
# 17.jpg
# 32.jpg
# 4.jpg
# 24.jpg
# 27.jpg
# 38.jpg
# 28.jpg
# 14.jpg
# 45.jpg
# 29.jpg
# 7.jpg
# 22.jpg
# 23.jpg
# 41.jpg
# 15.jpg
# 20.jpg""".strip().split()
# files = [f.strip() for f in files if f.strip()]

In [ ]:
files = """3.jpg
18.jpg
44.jpg
8.jpg
25.jpg
10.jpg
36.jpg
48.jpg
11.jpg
12.jpg
21.jpg
13.jpg
1.jpg
39.jpg
5.jpg
33.jpg
46.jpg
19.jpg
40.jpg
49.jpg
16.jpg
31.jpg
26.jpg
6.jpg
37.jpg
42.jpg
23.jpg
17.jpg
32.jpg
41.jpg
4.jpg
15.jpg
24.jpg
27.jpg
38.jpg
20.jpg
28.jpg
14.jpg
45.jpg
29.jpg
7.jpg
22.jpg
2.jpg
50.jpg
47.jpg
9.jpg
43.jpg
30.jpg
35.jpg
34.jpg
""".strip().split()
files = [f.strip() for f in files if f.strip()]

In [ ]:
with open("/home/nivg/data/DropGAN/template2.txt", "r") as f:
    txt = f.read()
# flag = '_mv'
flag = ''
folder = '/home/nivg/data/DropGAN/user_study/real/'
new_txt = ""
new_file = ""
# files = os.listdir(folder)
for i in range(0,len(files),2):
    file = files[i]
    print(file)
    new_txt = txt.replace("method_dolphins", "method_" + file.split('.')[0] + flag)
    new_txt = new_txt.replace("sigd_gt/dolphins.jpg", "places_gt/" + file)
    new_txt = new_txt.replace("dolphins", file.split('.')[0] + flag)
    new_txt = new_txt.replace("measures_dolphins", "measures_" + file.split('.')[0] + flag)
    new_txt = new_txt.replace("measures_dolphins", "measures_" + file.split('.')[0] + flag)
    new_txt = new_txt.replace("dolphins.jpg", file)
    new_txt = new_txt.replace(file.split('.')[0] + flag + '.jpg', file.split('.')[0] + '.jpg')
    file = files[i+1]
    print(file)
    new_txt = new_txt.replace("method_208", "method_" + file.split('.')[0] + flag)
    new_txt = new_txt.replace("sigd_gt/208.jpg", "places_gt/" + file)
    new_txt = new_txt.replace("208", file.split('.')[0] + flag)
    new_txt = new_txt.replace("measures_208", "measures_" + file.split('.')[0] + flag)
    new_txt = new_txt.replace("measures_208", "measures_" + file.split('.')[0] + flag)
    new_txt = new_txt.replace("208.jpg", file)
    new_txt = new_txt.replace("sigd_gt", "places_gt")
    new_txt = new_txt.replace("sigd_dg", "places_dg" + flag)
    new_txt = new_txt.replace("sigd_sg", "places_sg" + flag)
    new_txt = new_txt.replace("_GPNN", "_GPNN" + flag)
    new_txt = new_txt.replace("_source", "_source" + flag)
    new_txt = new_txt.replace("_SinGAN", "_SinGAN" + flag)
    new_txt = new_txt.replace(file.split('.')[0] + flag + '.jpg', file.split('.')[0] + '.jpg')
#     print(new_txt)
    new_file = new_file + new_txt + '\n'
# print(new_file)

with open("/home/nivg/data/DropGAN/sigd_html_places" + flag + "_new.txt", "w") as f:
    f.write(new_file)

In [ ]:
files = """3.jpg
18.jpg
44.jpg
8.jpg
34.jpg
25.jpg
10.jpg
35.jpg
36.jpg
30.jpg
48.jpg
11.jpg
12.jpg
43.jpg
21.jpg
13.jpg
1.jpg
39.jpg
2.jpg
5.jpg
33.jpg
46.jpg
50.jpg
19.jpg
40.jpg
47.jpg
49.jpg
16.jpg
31.jpg
9.jpg
26.jpg
6.jpg
37.jpg
42.jpg
17.jpg
32.jpg
4.jpg
24.jpg
27.jpg
38.jpg
28.jpg
14.jpg
45.jpg
29.jpg
7.jpg
22.jpg
23.jpg
41.jpg
15.jpg
20.jpg""".strip().split()
files = [f.strip() for f in files if f.strip()]

In [ ]:
txt.replace("places_gt/dolphins", "places_gt/ashdaz")